## Ejemplo: fit LRG con mocks de primera generación de caja cúbica

Ver [https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_challenge_y1](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_challenge_y1) para un ejemplo ejecutable

### Definir la likelihood

In [ ]:
from desilike.theories.galaxy_clustering import ShapeFitPowerSpectrumTemplate,\
                                                FOLPSTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood

template = ShapeFitPowerSpectrumTemplate(z=0.8)
theory = FOLPSTracerPowerSpectrumMultipoles(template=template)
observable = TracerPowerSpectrumMultipolesObservable(klim={0: [0.02, 0.18, 0.005], 2: [0.02, 0.18, 0.005]},
                                                     data='/global/cfs/cdirs/desi/cosmosim/KP45/MC/Clustering/AbacusSummit/CubicBox/LRG/Pk/Pre/jmena/nmesh_512/pypower_format/Pk_AbacusSummit_base_*.npy',
                                                     covariance='/global/cfs/cdirs/desi/cosmosim/KP45/MC/Clustering/EZmock/CubicBox/LRG/Pk/jmena/nmesh_512/pypower_format/Pk_EZmock_B2000G512Z0.8N8015724_b0.385d4r169c0.3_seed*.npy',
                                                     wmatrix='/global/cfs/cdirs/desi/users/adematti/desi_mock_challenge/FirstGenMocks/AbacusSummit/CubicBox/ELG/z1.100/window_nmesh512_los-x.npy',
                                                     kinlim=(0., 0.25),  # just to restrict input theory k's
                                                     theory=theory)
# Here we could give a list of several observables + joint covariance matrix
likelihood = ObservablesGaussianLikelihood(observables=[observable],
                                           scale_covariance=1. / 25.)
# We can also sum up independent likelihoods (likelihood1 + likelihood2)
# Call likelihood
likelihood(qpar=0.99, qper=1.01)

### Emular la teoría

Tip: Ejecutar con varios procesos MPI!

In [ ]:
from desilike.emulators import Emulator, TaylorEmulatorEngine, EmulatedCalculator

# theory.pt = Parte de teoría de perturbación, sólo depende de los parámetros del template (qpar, qper, df, dm)
emulator = Emulator(theory.pt, engine=TaylorEmulatorEngine(method='finite', order=4))
emulator.set_samples()
emulator.fit()
pt = emulator.to_calculator()
# emulator.save(emulator_fn); pt = EmulatedCalculator.load(emulator_fn)

# Reemplazar la parte de teoría de perturbación con la nueva!
theory.init.update(pt=pt)

### Sample

Tip: Ejecutar con varios procesos MPI!

In [ ]:
from desilike.samplers import EmceeSampler

sampler = EmceeSampler(likelihood, chains=8, nwalkers=40, seed=42, save_fn='chain_*.npy')
chains = sampler.run(min_iterations=2000, check={'max_eigen_gr': 0.03})

In [ ]:
### Re-cargar las chains y concatenarlas
from desilike.samples import Chain

chain = Chain.concatenate([Chain.load('chain_{:d}.npy'.format(i)).remove_burnin(0.5)[::10]
                           for i in range(8)])
# Si se desea convertir a formato GetDist:
chain.to_getdist()

### Usar las mediciones con compresión para hacer inferencia de las posteriors

In [ ]:
from desilike.observables.galaxy_clustering import ShapeFitCompressionObservable

observable = ShapeFitCompressionObservable(data=chain, covariance=chain,
                                           params=['qpar', 'qper', 'dm', 'df'], z=0.8)
likelihood = ObservablesGaussianLikelihood(observables=[observable])

sampler = EmceeSampler(likelihood, chains=8, nwalkers=40, seed=42, save_fn='chain_shapefit_*.npy')
chains = sampler.run(min_iterations=2000, check={'max_eigen_gr': 0.03})